In [31]:
import os
import pandas as pd

def extract_folder_info(folder_name):
    """Extract TIME and INSTANCE from the folder name."""
    try:
        first_underscore = folder_name.index('_')
        last_underscore = folder_name.rindex('_')
        TIME = folder_name[:first_underscore]
        INSTANCE = folder_name[first_underscore + 1:last_underscore]
        return TIME, INSTANCE
    except ValueError:
        return None, None

def extract_gap(result_file_path):
    """Extract GAP value from results.txt file."""
    GAP = 0
    if os.path.exists(result_file_path):
        try:
            with open(result_file_path, 'r') as result_file:
                for line in result_file:
                    if line.startswith("GapClosed:"):
                        GAP = float(line.split(':')[1].strip())
                        break
        except Exception as e:
            print(f"Error reading {result_file_path}: {e}")
    return GAP

def extract_solving_time(solving_stat_file_path):
    """Extract SOLVING_TIME value from solving_statistic.txt file."""
    SOLVING_TIME = 3700
    if os.path.exists(solving_stat_file_path):
        try:
            with open(solving_stat_file_path, 'r') as solving_stat_file:
                for line in solving_stat_file:
                    if line.startswith("Total Time"):
                        SOLVING_TIME = float(line.split(':')[1].strip())
                        break
        except Exception as e:
            print(f"Error reading {solving_stat_file_path}: {e}")
    return SOLVING_TIME

def extract_feasibility(result_file_path):
    """Extract FEASIBLE value from results.txt file."""
    FEASIBLE = False
    if os.path.exists(result_file_path):
        try:
            with open(result_file_path, 'r') as result_file:
                for line in result_file:
                    if line.startswith("DbgSolFeasible:"):
                        FEASIBLE = line.split(':')[1].strip().lower() == 'true'
                        break
        except Exception as e:
            print(f"Error reading {result_file_path}: {e}")
    return FEASIBLE

def process_folder(folder_path, folder_name, mode):
    """Process each folder to extract required information."""
    TIME, INSTANCE = extract_folder_info(folder_name)
    if TIME is None or INSTANCE is None:
        return None

    result_file_path = os.path.join(folder_path, 'results.txt')
    solving_stat_file_path = os.path.join(folder_path, 'solving_statistic.txt')

    GAP = extract_gap(result_file_path)
    SOLVING_TIME = extract_solving_time(solving_stat_file_path)
    FEASIBLE = extract_feasibility(result_file_path)

    return [INSTANCE, mode, GAP, SOLVING_TIME, FEASIBLE]

def fetchresult(main_dir):
    """Main function to iterate through directories and collect data."""
    data = []

    for mode in os.listdir(main_dir):
        mode_path = os.path.join(main_dir, mode)
        if os.path.isdir(mode_path):
            for folder_name in os.listdir(mode_path):
                folder_path = os.path.join(mode_path, folder_name)
                if os.path.isdir(folder_path):
                    folder_data = process_folder(folder_path, folder_name, mode)
                    if folder_data:
                        data.append(folder_data)

    df = pd.DataFrame(data, columns=['INSTANCE', 'MODE', 'GAP', 'TIME', 'FEASIBLE'])
    df['GAP'] = df['GAP'].astype(float)

    return df
    # Find and print pairs that a

In [32]:
main_dir = os.path.join(os.getcwd(), 'Data')
df = fetchresult(main_dir)

In [33]:
df[['INSTANCE', 'MODE']].drop_duplicates()

,INSTANCE,MODE
0,neos-3381206-awhea,vpcwsubscip
1,roll3000,vpcwsubscip
2,p200x1188c,vpcwsubscip
3,exp-1-500-5-5,vpcwsubscip
4,neos-3216931-puriri,vpcwsubscip
...,...,...
118,ran14x18-disj-8,vpc
119,p200x1188c,vpc
120,mik-250-20-75-4,vpc
121,seymour1,vpc


In [34]:
grouped_df = df.groupby(['INSTANCE', 'MODE']).size().reset_index(name='count')

# Filter pairs that appear more than once
result = grouped_df[grouped_df['count'] > 1]
result

,INSTANCE,MODE,count


In [35]:
# Pivot the DataFrame to the desired format
df_pivot = df.pivot(index='INSTANCE', columns='MODE', values=['GAP', 'TIME','FEASIBLE'])

# Flatten the multi-level columns
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]

gap_columns = [col for col in df_pivot.columns if 'GAP' in col]
df_pivot[gap_columns] = df_pivot[gap_columns].round(2)*100

In [36]:
df_pivot

,GAP_gomory,GAP_vpc,GAP_vpcwsubscip,TIME_gomory,TIME_vpc,TIME_vpcwsubscip,FEASIBLE_gomory,FEASIBLE_vpc,FEASIBLE_vpcwsubscip
INSTANCE,,,,,,,,,
30n20b8,9.68264,0.0,0.0,26.99,3657.02,3700.0,False,False,False
beasleyC3,1.17961,1.17961,1.17961,0.07,129.94,41.47,False,False,True
binkar10_1,6.75932,6.69134,6.69134,0.09,41.19,16.22,False,False,False
bnatt400,0.0,0.0,0.0,1.93,3531.65,3700.0,False,False,False
cod105,0.865617,0.437558,0.437558,4.13,163.12,134.9,False,False,True
cost266-UUE,27.2712,18.6341,21.7121,0.37,298.08,67.67,False,False,True
csched007,8.92488,3.38965,3.42123,0.25,237.8,169.84,False,False,False
exp-1-500-5-5,23.1601,21.7312,21.7312,0.06,36.2,14.98,False,False,True
gen-ip002,2.14825,0.906481,0.906481,0.01,0.17,3.22,False,False,True


KeyError: 'gomory_gap'

In [36]:
df_rounded

GAP                               TIME           \
MODE                       gomory           vpc   vpcwsubscip gomory      vpc   
INSTANCE                                                                        
30n20b8              9.682640e-02  0.000000e+00  0.000000e+00  26.99  3657.02   
beasleyC3            1.179610e-02  1.179610e-02  2.811230e-01   0.07   129.94   
binkar10_1           6.759320e-02  6.691340e-02  6.272730e-02   0.09    41.19   
bnatt400             0.000000e+00  0.000000e+00  0.000000e+00   1.93  3531.65   
cod105               8.656170e-03  4.375580e-03  0.000000e+00   4.13   163.12   
cost266-UUE          2.727120e-01  1.863410e-01  2.110370e-01   0.37   298.08   
csched007            8.924880e-02  3.389650e-02  3.227820e-03   0.25   237.80   
exp-1-500-5-5        2.316010e-01  2.173120e-01  2.173120e-01   0.06    36.20   
gen-ip002            2.148250e-02  9.064810e-03  4.108320e-03   0.01     0.17   
glass4               0.000000e+00 -8.881780e-15  0.000000e+00   0.05     2.67   
graph20-20-1rand     0.000000e+00  0.000000e+00  0.000000e+00   0.39     4.53   
graphdraw-domain     1.149490e-03  0.000000e+00  0.000000e+00   0.08     2.63   
icir97_tension       2.015170e-01  2.015170e-01  3.749690e-02   0.47   265.08   
mc11                 4.637900e-03  7.964280e-03  5.085280e-03   0.13  1016.71   
mcsched              4.081780e-04  0.000000e+00  0.000000e+00   1.04  3700.00   
mik-250-20-75-4      5.383000e-01  5.328460e-01  5.328460e-01   0.05     3.26   
milo-v12-6-r2-40-1   0.000000e+00  0.000000e+00  0.000000e+00   0.56  3700.00   
n5-3                 1.525170e-01  1.320470e-01  3.136090e-01   0.14    49.78   
neos-1582420         5.328520e-02  1.845200e-02  0.000000e+00   2.42  2333.74   
neos-3083819-nubu    4.236710e-01  2.312110e-03  1.033400e-02   0.43   120.86   
neos-3216931-puriri  1.684130e-02  0.000000e+00  0.000000e+00  38.83  3700.00   
neos-3381206-awhea   1.554310e-15  0.000000e+00  2.997600e-15   0.16   601.10   
neos-3627168-kasai   1.421100e-01  1.421100e-01  3.510450e-03   0.15   151.74   
neos-911970          1.110220e-16  0.000000e+00  0.000000e+00   0.14    16.13   
neos17               4.778510e-06  0.000000e+00  1.965360e-03   0.08     0.10   
neos5                1.157410e-01  8.333330e-02  8.333330e-02   0.02     0.42   
ns1208400            0.000000e+00  0.000000e+00  0.000000e+00   2.24  2877.51   
ns1830653            1.430070e-01  2.223210e-02  0.000000e+00   1.13    98.72   
nu25-pr12            5.257700e-01  4.432720e-02  3.198260e-01   0.43   222.52   
p200x1188c           4.474480e-03  4.761820e-03  4.761820e-03   0.06     6.74   
pg                   2.429180e-01  2.429880e-01  2.429880e-01   0.07   168.22   
pg5_34               2.344960e-01  2.344960e-01  2.344960e-01   0.08   152.08   
ran14x18-disj-8      2.778530e-03  1.326700e-03  0.000000e+00   0.19    40.99   
reblock115           7.166100e-02  9.966030e-02  1.264510e-01   2.46  2828.68   
rococoC10-001000     2.900880e-01  1.453290e-01  2.569880e-02   0.32   375.33   
roll3000             1.213880e-01  2.480680e-03  3.232820e-04   0.56    95.91   
seymour1             1.346670e-01  1.346570e-01  1.168990e-01   1.10   337.84   
sp150x300d           1.466360e-02  1.097780e-02  1.586750e-01   0.03     2.55   
timtab1              1.255630e-01  1.124410e-01  1.777960e-01   0.03     2.33   
tr12-30             -3.015460e+00 -2.847930e+00 -4.936470e+00   0.08    97.52   
traininstance6       0.000000e+00  0.000000e+00  0.000000e+00   4.94   223.94   
uct-subprob          1.480140e-02  1.470880e-02  9.823220e-03   1.15  2993.58   

                                 
MODE                vpcwsubscip  
INSTANCE                         
30n20b8                 4414.77  
beasleyC3                241.77  
binkar10_1               204.52  
bnatt400                3700.00  
cod105                  3700.00  
cost266-UUE              143.49  
csched007                470.88  
exp-1-500-5-5             28.21  
gen-ip002 